In [1]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

import timeit
import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing

%matplotlib inline

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [3]:
data = pd.read_csv("housesalesprediction/kc_house_data.csv")
data.shape

(21613, 21)

In [4]:
feature_cols = ['bedrooms', 'bathrooms', 'sqft_living','sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
             'sqft_above', 'sqft_basement','sqft_living15', 'sqft_lot15']
cont_cols = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement','sqft_living15','sqft_lot15']
cat_cols = ['waterfront', 'view', 'condition', 'grade']
target_col = 'price'

In [5]:
X = data[feature_cols]
Y = data[target_col]

In [6]:
def remove_outlier(df_in):
    for col_name in df_in.columns:
        if col_name in cont_cols:
            q1 = df_in[col_name].quantile(0.25)
            q3 = df_in[col_name].quantile(0.75)
            iqr = q3-q1 #Interquartile range
            fence_low  = q1-1.5*iqr
            fence_high = q3+1.5*iqr
            df_in[col_name] = df_in[col_name].clip(fence_low,fence_high)
        else:
            pass
    return df_in

In [7]:
X_wo = remove_outlier(X.copy())

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_wo, Y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(15129, 13)
(6484, 13)
(15129,)
(6484,)


In [9]:
# Scaling for KNN

scaler = preprocessing.MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train[cont_cols]), columns=cont_cols, index = X_train.index)
test_scaled = pd.DataFrame(scaler.transform(X_test[cont_cols]), columns=cont_cols, index = X_test.index)

X_train = pd.concat([train_scaled, X_train[cat_cols]], axis =1)
X_test = pd.concat([test_scaled, X_test[cat_cols]], axis =1)

In [10]:
# import the regressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [11]:
# PNN - Ensembling CART and KNN

start_time = timeit.default_timer()
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0, min_samples_leaf = 20, max_depth = 4)
  
# fit the regressor with X and Y data
regressor.fit(X_train, Y_train)
y_train_predict = regressor.predict(X_train)
y_test_predict = regressor.predict(X_test)

df_train = pd.concat([X_train,Y_train], axis = 1)
df_train['Tree_pred'] = y_train_predict
df_train.loc[:,'Leaf_node'] = regressor.apply(X_train) # To get leaf node for each training sample

df_test = pd.concat([X_test,Y_test], axis = 1)
df_test['Tree_pred'] = y_test_predict
df_test.loc[:,'Leaf_node'] = regressor.apply(X_test) # To get leaf node for each test sample

df_final = pd.DataFrame()
for i in df_test['Leaf_node'].unique():
    X_train = df_train[df_train['Leaf_node']==i]
    X_test = df_test[df_test['Leaf_node']==i]
    
    if(X_train.shape[0]>10):
        neigh = KNeighborsRegressor(n_neighbors=5)
        neigh.fit(X_train[feature_cols],X_train[target_col])
        X_test.loc[:,'pred'] = neigh.predict(X_test[feature_cols])
    elif(X_train.shape[0]>=3):
        neigh = KNeighborsRegressor(n_neighbors=3)
        neigh.fit(X_train[feature_cols],X_train[target_col])
        X_test.loc[:,'pred'] = neigh.predict(X_test[feature_cols])        
    else:
        X_test.loc[:,'pred'] = regressor.predict(X_test[feature_cols])
        
    df_final = pd.concat([df_final,X_test], axis = 0)

print("Time elapsed in seconds",timeit.default_timer() - start_time)

Time elapsed in seconds 2.0287273000000003


In [12]:
# RMSE for Decision Tree Predictions

(np.sqrt(mean_squared_error(df_final[target_col], df_final['Tree_pred'])))

234357.19250368702

In [13]:
# RMSE for PNN predictions

(np.sqrt(mean_squared_error(df_final[target_col], df_final['pred'])))

220270.5054191276

In [14]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
# MAPE for Decision Tree Predictions
mean_absolute_percentage_error(df_final[target_col], df_final['Tree_pred'])

32.08694592006732

In [16]:
# MAPE for PNN prediction
mean_absolute_percentage_error(df_final[target_col], df_final['pred'])

26.61722748426939

In [17]:
# Standard deviation in prediction of Decision Tree and PNN
df_final['Tree_pred'].std(), df_final['pred'].std()

(289361.889141703, 313631.288087317)

### Random Forest

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_wo, Y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(15129, 13)
(6484, 13)
(15129,)
(6484,)


In [19]:
start_time = timeit.default_timer()

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

regr = RandomForestRegressor(max_depth=3, random_state=0,
                             n_estimators=100)
regr.fit(X_train, Y_train)

# model evaluation for testing set
y_test_predict = regr.predict(X_test)

print("Time elapsed in seconds",timeit.default_timer() - start_time)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

print('MAPE for testing set is')
mean_absolute_percentage_error(Y_test, y_test_predict)

Time elapsed in seconds 3.1589329000000004
The model performance for testing set
--------------------------------------
RMSE is 239401.49754966956
MAPE for testing set is


33.158558855620804


### KNN Regression

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_wo, Y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(15129, 13)
(6484, 13)
(15129,)
(6484,)


In [21]:
# Scaling for KNN

scaler = preprocessing.MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train[cont_cols]), columns=cont_cols, index = X_train.index)
test_scaled = pd.DataFrame(scaler.transform(X_test[cont_cols]), columns=cont_cols, index = X_test.index)

X_train = pd.concat([train_scaled, X_train[cat_cols]], axis =1)
X_test = pd.concat([test_scaled, X_test[cat_cols]], axis =1)

In [22]:
start_time = timeit.default_timer()

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

neigh = KNeighborsRegressor(n_neighbors=5, metric="euclidean")
neigh.fit(X_train, Y_train)

# model evaluation for testing set
y_test_predict = neigh.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
print("Time elapsed in seconds",timeit.default_timer() - start_time)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

print('MAPE for testing set is')
mean_absolute_percentage_error(Y_test, y_test_predict)

Time elapsed in seconds 1.4480021
The model performance for testing set
--------------------------------------
RMSE is 216766.55380178537
MAPE for testing set is


26.40565990234704